# Weclome to they berry processing pipeline!

This pipeline is capable of intaking .cdf files, preprocessing the data which consist of noise reduction, baseline correction and peak identification. This step also produces the .txt files which are used to generate chemical profiles by querying the NIST database using the Windows based MSSEARCH tool.
Once peaks have been identified, they are aligned across the varios berry sample to produce a consensus based alignment .csv file. The alignment .csv file is then used to generate a heirarchical cluster analysis.

## The preprocessing script (berryPipeline.py)

This script output three file types:
expr - Contains peak data used in the alignment analysis
csv/txt - Contains MS data to be queried by the NIST MSSEAECH tool.


Command line variables:
1. .cdf file directory
2. String to break name one (Generally directory where files are located) 
3. Points (Window size)
4. Scan (Number of windows to average)
5. Percentage threshold 
6. Minimum of ions required
7. .csv file storage directory
8. .expr file storage directory


In [3]:
%run -i berryPipe.py \
/workdir2/shared_folder/cpowell/fielTmp/all/jup/ \
/2016_blueberry_gwas/ \
140 \
25 \
3 \
3 \
/workdir2/shared_folder/cpowell/expr/jup_nist/ \
/workdir2/shared_folder/cpowell/expr/jup_expr/ 


('CDF file directory:', '/workdir2/shared_folder/cpowell/fielTmp/all/jup/')
('split:', '/2016_blueberry_gwas/')
('Points:', 140)
('Scans:', 25)
('Percent:', 3)
('num. of ions:', 3)
('Name_tag:', 'p140s25%3n3')
('Storage directory (csv):', '/workdir2/shared_folder/cpowell/expr/jup_nist/')
('Storage dir (expr):', '/workdir2/shared_folder/cpowell/expr/jup_expr/')
('n1', ['Blue-16-A-1008_518.cdf', 'Blue-16-A-1007_269.cdf', 'Blue-16-A-1005_403.cdf', 'Blue-16-A-1008_392.cdf', 'Blue-16-A-1007_62.cdf', 'Blue-16-A-1005_50.cdf', 'Blue-16-A-1008_145.cdf', 'Blue-16-A-1007_457.cdf', 'Blue-16-A-1005_295.cdf'])
('name=', 'Blue-16-A-1008_518.cdf')
 -> Reading netCDF file '/workdir2/shared_folder/cpowell/fielTmp/all/jup/Blue-16-A-1008_518.cdf'
[ 35.93493271  36.82106018  38.10810089 ... 218.32415771 219.34289551
 219.91921997]


KeyboardInterrupt: 

## The Alignment script( Salign.py)

In this version of the alignment script each sample is aligned without any grouping constaints. Potential groupings might be breeds or berry replicates. 

Input variables:
1. expr directory
2. alignment csv output directory 


In [1]:
%env 

{'BASH_FUNC_module()': '() {  eval `/usr/bin/modulecmd bash $*`\n}',
 'CLICOLOR': '1',
 'CONDA_DEFAULT_ENV': 'beta',
 'CONDA_EXE': '/workdir2/cpowell/conda/miniconda2/bin/conda',
 'CONDA_PREFIX': '/workdir2/cpowell/conda/miniconda2/envs/beta',
 'CONDA_PREFIX_1': '/workdir2/cpowell/conda/miniconda2',
 'CONDA_PROMPT_MODIFIER': '(beta) ',
 'CONDA_PYTHON_EXE': '/workdir2/cpowell/conda/miniconda2/bin/python',
 'CONDA_SHLVL': '2',
 'CVS_RSH': 'ssh',
 'GIT_PAGER': 'cat',
 'G_BROKEN_FILENAMES': '1',
 'HISTCONTROL': 'ignoredups',
 'HISTSIZE': '1000',
 'HOME': '/home/cpowell',
 'HOSTNAME': 'sjcbiohpc01.driscolls.com',
 'JPY_PARENT_PID': '13427',
 'LANG': 'en_US.UTF-8',
 'LESSOPEN': '||/usr/bin/lesspipe.sh %s',
 'LOADEDMODULES': '',
 'LOGNAME': 'cpowell',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=01;05;37;41:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arj=01;31:*.taz=01;31:*.lzh=01;31:

In [2]:
!env

HOSTNAME=sjcbiohpc01.driscolls.com
SHELL=/bin/bash
TERM=xterm-color
CLICOLOR=1
HISTSIZE=1000
SSH_CLIENT=10.10.96.103 50167 22
CONDA_SHLVL=2
CONDA_PROMPT_MODIFIER=(beta) 
QTDIR=/usr/lib64/qt-3.3
QTINC=/usr/lib64/qt-3.3/include
SSH_TTY=/dev/pts/1
USER=cpowell
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=01;05;37;41:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arj=01;31:*.taz=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lz=01;31:*.xz=01;31:*.bz2=01;31:*.tbz=01;31:*.tbz2=01;31:*.bz=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.rar=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.jpg=01;35:*.jpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:*.tif=01;35:*.tiff=01;35:*.png=01;35:*.svg=01;35:*.svgz=01;35:*.mng=01;35:*.pcx=01;35:*.mov=01;35:*.mp

In [ ]:
!export LD_LIBRARY_PATH=/opt/glibc-2.14/lib

In [ ]:
%set_env LD_LIBRARY_PATH=/opt/glibc-2.14/lib

In [ ]:
%export LD_LIBRARY_PATH=/opt/glibc-2.14/lib

In [ ]:
!env

In [ ]:
%env

In [4]:
%run -i Salign.py \
/workdir2/shared_folder/cpowell/expr/jup_expr/ \
/workdir2/shared_folder/cpowell/expr/jup_nist/

ImportError: /lib64/libc.so.6: version `GLIBC_2.14' not found (required by /workdir2/cpowell/conda/miniconda2/envs/beta/lib/python2.7/site-packages/Pycluster/cluster.so)

## The HCA generator script (scrap.py)

Using the aligment .csv file produced in the previous step this script generates a distance matrix to compare each alignment profile in order to generate an HCA.

Input variables:
1. Alignment .csv file


In [ ]:
%run -i scrap.py \
/workdir2/shared_folder/cpowell/expr/jup_nist/2016BG_area.csv
